# Assignment 5: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: Aleesha Zainab
#### Student ID: 04162013021

Date: 06/15/2024

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used:
* pandas
* re
* numpy
* nltk




## Introduction
This task focuses on performing basic text pre-processing on a dataset of job advertisements, particularly on the description field. Here's a description of the steps involved:

1. **Extraction of Information**: Extract information from each job advertisement and preprocess the description.

2. **Tokenization**: Tokenize each job advertisement description using a specified regular expression pattern.

3. **Lowercasing**: Convert all words to lowercase to ensure uniformity.

4. **Removal of Short Words**: Remove words with a length less than 2.

5. **Stopword Removal**: Remove stopwords using a provided stop words list.

6. **Term Frequency-Based Removal**: Remove words that appear only once in the document collection, based on term frequency.

7. **Top Frequent Word Removal**: Remove the top 50 most frequent words based on document frequency.

8. **Saving Preprocessed Job Ads**: Save all preprocessed job advertisement texts and information in text file(s).

9. **Vocabulary Building**: Build a vocabulary of the cleaned job advertisement descriptions and save it in a text file.

Note:
- Removed words (including those removed in steps 4, 5, 6, and 7) are excluded from the generated vocabulary.
- The vocabulary file must contain unigram vocabulary, sorted in alphabetical order, with integer index values starting from 0.

Overall, this task ensures that the job advertisement descriptions are preprocessed effectively for further analysis, with specific steps taken to clean and organize the text data.

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries

In [ ]:
import os
import re
from collections import Counter
from collections import defaultdict
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1.1 Examining and loading data



In [ ]:
data_dir = '/content/drive/MyDrive/Assignment5 data/data'
categories = ['Accounting_Finance', 'Engineering', 'Healthcare_Nursing', 'Sales']


In [ ]:
job_ads = []
web_indexes = []
def load_job_ads(data_dir, categories):
    for category in categories:
        category_path = os.path.join(data_dir, category)
        for filename in os.listdir(category_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(category_path, filename)
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    if len(lines) > 1:
                        title = lines[0].strip()
                        web_index = lines[1].strip().split(":")[-1].strip()
                        description = " ".join(lines[2:]).strip()
                        job_ads.append({
                            'category': category,
                            'filename': filename,
                            'title': title,
                            'web_index': web_index,
                            'description': description
                        })

                        web_indexes.append(web_index)
data_dir = '/content/drive/MyDrive/Assignment5 data/data'


In [ ]:
load_job_ads(data_dir, categories)
job_ads_df = pd.DataFrame(job_ads)
y_labels = job_ads_df['category']
y_labels_csv_path = '/content/drive/MyDrive/Assignment5 data/y_labels.csv'
y_labels.to_csv(y_labels_csv_path, index=False, header=['category'])



In [ ]:
import csv
csv_file_path = os.path.join(os.getcwd(), '/content/drive/MyDrive/web_index.csv')
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['web_index'])
    for web_index in web_indexes:
        writer.writerow([web_index])

print(f"Web indexes have been saved to {csv_file_path}")

Web indexes have been saved to /content/drive/MyDrive/web_index.csv


### 1.2 Pre-processing data
Perform the required text pre-processing steps.

In [ ]:
token_pattern = re.compile(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")

In [ ]:
def preprocess_text(text):
    tokens = token_pattern.findall(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if len(token) > 1]
    return tokens


In [ ]:
for ad in job_ads:
    ad['tokens'] = preprocess_text(ad['description'])

In [ ]:
stop_words_file = '/content/drive/MyDrive/Assignment5 data/stopwords_en.txt'
with open(stop_words_file, 'r') as f:
    stop_words = set(f.read().splitlines())


In [ ]:
print(f"Total number of job ads: {len(job_ads)}")
for category in categories:
    num_ads = sum(1 for ad in job_ads if ad['category'] == category)
    print(f"Number of ads in {category}: {num_ads}")

print("\nSample job ad:")
print(job_ads[0])

Total number of job ads: 776
Number of ads in Accounting_Finance: 191
Number of ads in Engineering: 231
Number of ads in Healthcare_Nursing: 198
Number of ads in Sales: 156

Sample job ad:
{'category': 'Accounting_Finance', 'filename': 'Job_00310.txt', 'title': 'Title: Commercial Team Leader', 'web_index': '71293696', 'description': 'Company: Hillman Saunders\n Description: Our client, a highly regarded and professional insurance brokerage based in West Sussex, is looking to recruit a team leader/manager for their commercial insurance schemes team. This is an internal sales operations role managing **** telephone account handlers and **** admin support staff that service the bulk of their SME type clients in the charity sector. It s a fairly rounded role with candidates needing to have all round people/team management experience and excellent customer /sales skills to help drive up performance of the team. Ideally candidates should have schemes experience where processes are quite key 

# remove stopwords

In [ ]:
def remove_stopwords(tokens, stop_words):
    return [token for token in tokens if token not in stop_words]

In [ ]:
for ad in job_ads:
    ad['tokens'] = remove_stopwords(ad['tokens'], stop_words)

# Count term frequencies

In [ ]:
term_freq = Counter(token for ad in job_ads for token in ad['tokens'])

# Remove rare words

In [ ]:
def remove_rare_words(tokens, term_freq, min_freq=2):
    return [token for token in tokens if term_freq[token] >= min_freq]

In [ ]:
for ad in job_ads:
    ad['tokens'] = remove_rare_words(ad['tokens'], term_freq)

print("\nTokens after removing rare words for sample job ad:")
print(job_ads[0]['tokens'])


Tokens after removing rare words for sample job ad:
['company', 'hillman', 'saunders', 'description', 'client', 'highly', 'regarded', 'professional', 'insurance', 'brokerage', 'based', 'west', 'sussex', 'recruit', 'team', 'leader', 'manager', 'commercial', 'insurance', 'schemes', 'team', 'internal', 'sales', 'operations', 'role', 'managing', 'telephone', 'account', 'handlers', 'admin', 'support', 'staff', 'service', 'bulk', 'sme', 'type', 'clients', 'charity', 'sector', 'fairly', 'rounded', 'role', 'candidates', 'needing', 'round', 'people', 'team', 'management', 'experience', 'excellent', 'customer', 'sales', 'skills', 'drive', 'performance', 'team', 'ideally', 'candidates', 'schemes', 'experience', 'processes', 'key', 'ability', 'review', 'improve', 'current', 'processes', 'improve', 'team', 'efficiency', 'monitoring', 'team', 'compliance', 'broker', 'client', 'commercial', 'classes', 'business', 'broad', 'technical', 'product', 'knowledge', 'needed', 'provide', 'st', 'line', 'suppo

#Identify top 50 most frequent words

In [ ]:
doc_freq = defaultdict(int)
for ad in job_ads:
    unique_tokens = set(ad['tokens'])
    for token in unique_tokens:
        doc_freq[token] += 1

top_50_words = {word for word, count in Counter(doc_freq).most_common(50)}


# Function to remove frequent words

In [ ]:
def remove_most_frequent_words(tokens, top_50_words):
    return [token for token in tokens if token not in top_50_words]

In [ ]:
for ad in job_ads:
    ad['tokens'] = remove_most_frequent_words(ad['tokens'], top_50_words)

In [ ]:
preprocessed_dir = '/content/drive/MyDrive/Assignment5 data/preprocessed_data'
os.makedirs(preprocessed_dir, exist_ok=True)

# Save preprocessed job ads

In [ ]:
for ad in job_ads:
    preprocessed_file_path = os.path.join(preprocessed_dir, ad['filename'])
    with open(preprocessed_file_path, 'w') as f:
        f.write(" ".join(ad['tokens']))

print("Preprocessed job ads saved.")

Preprocessed job ads saved.


In [ ]:
vocabulary = set(token for ad in job_ads for token in ad['tokens'])

In [ ]:
sorted_vocabulary = sorted(vocabulary)
vocab_with_index = {word: idx for idx, word in enumerate(sorted_vocabulary)}


## Saving required outputs
Save the vocabulary in vocab.txt.
- vocab.txt

In [ ]:
vocabulary_file = os.path.join(preprocessed_dir, 'vocab.txt')
with open(vocabulary_file, 'w') as f:
    for word, idx in vocab_with_index.items():
        f.write(f"{word}:{idx}\n")

print("Vocabulary saved.")


Vocabulary saved.


## Summary
In this task, I tackled basic text pre-processing for a job advertisement dataset. I performed steps like tokenization, lowercasing, removing short words and stopwords, and eliminating rare and frequent terms. These steps ensure that the text data is cleaned and organized for further analysis. I then saved the preprocessed job ads and built a vocabulary, essential for subsequent tasks. This thorough pre-processing lays the groundwork for more advanced natural language processing tasks on the dataset.